<a href="https://colab.research.google.com/github/Offliners/writeup/blob/main/HW7/homework7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework 7 - Bert (Question Answering)**

If you have any questions, feel free to email us at ntu-ml-2021spring-ta@googlegroups.com



Slide:    [Link](https://docs.google.com/presentation/d/1aQoWogAQo_xVJvMQMrGaYiWzuyfO0QyLLAhiMwFyS2w)　Kaggle: [Link](https://www.kaggle.com/c/ml2021-spring-hw7)　Data: [Link](https://drive.google.com/uc?id=1znKmX08v9Fygp-dgwo7BKiLIf2qL1FH1)




## Task description
- Chinese Extractive Question Answering
  - Input: Paragraph + Question
  - Output: Answer

- Objective: Learn how to fine tune a pretrained model on downstream task using transformers

- Todo
    - Fine tune a pretrained chinese BERT model
    - Change hyperparameters (e.g. doc_stride)
    - Apply linear learning rate decay
    - Try other pretrained models
    - Improve preprocessing
    - Improve postprocessing
- Training tips
    - Automatic mixed precision
    - Gradient accumulation
    - Ensemble

- Estimated training time (tesla t4 with automatic mixed precision enabled)
    - Simple: 8mins
    - Medium: 8mins
    - Strong: 25mins
    - Boss: 2hrs
  

## Download Dataset

In [1]:
# For this HW, K80 < P4 < T4 < P100 <= T4(fp16) < V100
!nvidia-smi

Fri May 21 11:44:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Download link 1
!gdown --id '1znKmX08v9Fygp-dgwo7BKiLIf2qL1FH1' --output hw7_data.zip

# Download Link 2 (if the above link fails) 
# !gdown --id '1pOu3FdPdvzielUZyggeD7KDnVy9iW1uC' --output hw7_data.zip

!unzip -o hw7_data.zip

Downloading...
From: https://drive.google.com/uc?id=1znKmX08v9Fygp-dgwo7BKiLIf2qL1FH1
To: /content/hw7_data.zip
7.71MB [00:00, 24.5MB/s]
Archive:  hw7_data.zip
  inflating: hw7_dev.json            
  inflating: hw7_test.json           
  inflating: hw7_train.json          


## Install transformers

Documentation for the toolkit:　https://huggingface.co/transformers/

In [3]:
# You are allowed to change version of transformers or use other toolkits
!pip install transformers==4.5.0

     |████████████████████████████████| 2.2MB 7.3MB/s 
     |████████████████████████████████| 901kB 46.9MB/s 
     |████████████████████████████████| 3.3MB 50.7MB/s 


## Import Packages

In [4]:
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset, ConcatDataset
from transformers import AdamW, BertForQuestionAnswering, BertTokenizerFast
from transformers import AutoTokenizer, AutoModel
from tqdm.auto import tqdm
from transformers import get_linear_schedule_with_warmup
from random import randint

device = "cuda" if torch.cuda.is_available() else "cpu"

# Fix random seed for reproducibility
def same_seeds(seed):
	  torch.manual_seed(seed)
	  if torch.cuda.is_available():
		    torch.cuda.manual_seed(seed)
		    torch.cuda.manual_seed_all(seed)
	  np.random.seed(seed)
	  random.seed(seed)
	  torch.backends.cudnn.benchmark = False
	  torch.backends.cudnn.deterministic = True
same_seeds(0)

In [5]:
# Change "fp16_training" to True to support automatic mixed precision training (fp16)	
fp16_training = True

if fp16_training:
    !pip install accelerate==0.2.0
    from accelerate import Accelerator
    accelerator = Accelerator(fp16=True)
    device = accelerator.device

# Documentation for the toolkit:  https://huggingface.co/docs/accelerate/

     |████████████████████████████████| 51kB 3.3MB/s 


## Load Model and Tokenizer




 

In [6]:
# model = BertForQuestionAnswering.from_pretrained("bert-base-chinese").to(device)
# tokenizer = BertTokenizerFast.from_pretrained("bert-base-chinese")

from transformers import AutoTokenizer, AutoModelForQuestionAnswering
  
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")

model = AutoModelForQuestionAnswering.from_pretrained("hfl/chinese-macbert-base")

# You can safely ignore the warning message (it pops up because new prediction heads for QA are initialized randomly)

Some weights of the model checkpoint at hfl/chinese-macbert-base were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the 

## Read Data

- Training set: 26935 QA pairs
- Dev set: 3523  QA pairs
- Test set: 3492  QA pairs

- {train/dev/test}_questions:	
  - List of dicts with the following keys:
   - id (int)
   - paragraph_id (int)
   - question_text (string)
   - answer_text (string)
   - answer_start (int)
   - answer_end (int)
- {train/dev/test}_paragraphs: 
  - List of strings
  - paragraph_ids in questions correspond to indexs in paragraphs
  - A paragraph may be used by several questions 

In [7]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]

train_questions, train_paragraphs = read_data("hw7_train.json")
dev_questions, dev_paragraphs = read_data("hw7_dev.json")
test_questions, test_paragraphs = read_data("hw7_test.json")

## Tokenize Data

In [8]:
# Tokenize questions and paragraphs separately
# 「add_special_tokens」 is set to False since special tokens will be added when tokenized questions and paragraphs are combined in datset __getitem__ 

train_questions_tokenized = tokenizer([train_question["question_text"] for train_question in train_questions], add_special_tokens=False)
dev_questions_tokenized = tokenizer([dev_question["question_text"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False) 

train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False)
dev_paragraphs_tokenized = tokenizer(dev_paragraphs, add_special_tokens=False)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False)

# You can safely ignore the warning message as tokenized sequences will be futher processed in datset __getitem__ before passing to model

## Dataset and Dataloader

In [12]:
class QA_Dataset(Dataset):
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 100
        self.max_paragraph_len = 400
        
        ##### TODO: Change value of doc_stride #####
        self.doc_stride = 128

        # Input sequence length = [CLS] + question + [SEP] + paragraph + [SEP]
        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        tokenized_question = self.tokenized_questions[idx]
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]

        ##### TODO: Preprocessing #####
        # Hint: How to prevent model from learning something it should not learn

        if self.split == "train":
            # Convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph  
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])

            # A single window is obtained by slicing the portion of paragraph containing the answer
            mid = (answer_start_token + answer_end_token) // 2
            paragraph_start = max(0, min(mid - self.max_paragraph_len // 2, len(tokenized_paragraph) - self.max_paragraph_len))
            paragraph_end = paragraph_start + self.max_paragraph_len

            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102] 
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]		
            
            # Convert answer's start/end positions in tokenized_paragraph to start/end positions in the window  
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start

            # Pad sequence and obtain inputs to model 
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        # Validation/Testing
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []
            
            # Paragraph is split into several windows, each with start positions separated by step "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride):
                
                # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102]
                
                # Pad sequence and obtain inputs to model
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
                
                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)
            
            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list)

    def padding(self, input_ids_question, input_ids_paragraph):
        # Pad zeros if sequence length is shorter than max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # Indices of input sequence tokens in the vocabulary
        input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
        # Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
        token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        # Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len
        
        return input_ids, token_type_ids, attention_mask

train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
dev_set = QA_Dataset("dev", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)

train_batch_size = 8

# Note: Do NOT change batch size of dev_loader / test_loader !
# Although batch size=1, it is actually a batch consisting of several windows from the same QA pair
# temp_set = ConcatDataset([train_set, dev_set])
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)

## Function for Evaluation

In [13]:
def evaluate(data, output, index, split):
    ##### TODO: Postprocessing #####
    # There is a bug and room for improvement in postprocessing 
    # Hint: Open your prediction file to see what is wrong 
    
    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]
    
    for k in range(num_of_windows):
        # Obtain answer by choosing the most probable start position / end position
        start_prob, start_index = torch.max(output.start_logits[k], dim=0)
        end_prob, end_index = torch.max(output.end_logits[k], dim=0)
        
        # Probability of answer is calculated as sum of start_prob and end_prob
        prob = start_prob + end_prob
        
        # Replace answer if calculated probability is larger than previous windows
        if prob > max_prob:
            max_prob = prob
            # Convert tokens to chars (e.g. [1920, 7032] --> "大 金")
            answer = tokenizer.decode(data[0][0][k][start_index : end_index + 1])

            if answer.find('[UNK]') != -1:
                if split == 'dev':
                    print(f'ID {index} Before : {answer}')
                    start = start_index - min(len(dev_questions[index]['question_text']), 50) - 2 + k * 128
                    end = end_index - min(len(dev_questions[index]['question_text']), 50) - 2 + k * 128
                    start_target = dev_paragraphs_tokenized[dev_questions[index]['paragraph_id']].offsets[start]
                    end_target = dev_paragraphs_tokenized[dev_questions[index]['paragraph_id']].offsets[end]
                    target = dev_paragraphs[dev_questions[index]['paragraph_id']][start_target[0]: end_target[1]]
                    answer = target
                    print(f'ID {index} After : {answer}')
                if split == 'test':
                    print(f'ID {index} Before : {answer}')
                    start = start_index - min(len(test_questions[index]['question_text']), 50) - 2 + k * 128
                    end = end_index - min(len(test_questions[index]['question_text']), 50) - 2 + k * 128
                    start_target = test_paragraphs_tokenized[test_questions[index]['paragraph_id']].offsets[start]
                    end_target = test_paragraphs_tokenized[test_questions[index]['paragraph_id']].offsets[end]
                    target = test_paragraphs[test_questions[index]['paragraph_id']][start_target[0]: end_target[1]]
                    answer = target
                    print(f'ID {index} After : {answer}')

    answer = answer.replace('[CLS]','')                   
    answer = answer.replace('[SEP]','')
    # Remove spaces in answer (e.g. "大 金" --> "大金")
    return answer.replace(' ','')

## Training

In [14]:
num_epoch = 3
validation = True
logging_step = 100
learning_rate = 3e-5
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_loader) * num_epoch
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=1000, num_training_steps=total_steps)

if fp16_training:
    model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader) 

accum_iter = 4

model.train()

print("Start Training ...")

for epoch in range(num_epoch):
    step = 1
    train_loss = train_acc = 0

    for batch_idx, data in enumerate(tqdm(train_loader)):	
        # Load all data into GPU
        data = [i.to(device) for i in data]
        
        with torch.set_grad_enabled(True):
            # Model inputs: input_ids, token_type_ids, attention_mask, start_positions, end_positions (Note: only "input_ids" is mandatory)
            # Model outputs: start_logits, end_logits, loss (return when start_positions/end_positions are provided)  
            output = model(input_ids=data[0], token_type_ids=data[1], attention_mask=data[2], start_positions=data[3], end_positions=data[4])

            # Choose the most probable start position / end position
            start_index = torch.argmax(output.start_logits, dim=1)
            end_index = torch.argmax(output.end_logits, dim=1)
            
            # Prediction is correct only if both start_index and end_index are correct
            train_acc += ((start_index == data[3]) & (end_index == data[4])).float().mean()
            train_loss += output.loss
            
            output.loss = output.loss / accum_iter

            if fp16_training:
                accelerator.backward(output.loss)
            else:
                output.loss.backward()
            
            if ((batch_idx + 1) % accum_iter == 0) or (batch_idx + 1 == len(train_loader)):
                optimizer.step()
                optimizer.zero_grad()
            
            step += 1

            ##### TODO: Apply linear learning rate decay #####
            scheduler.step()

            # Print training loss and accuracy over past logging step
            if step % logging_step == 0:
                print(f"Epoch {epoch + 1} | Step {step} | loss = {train_loss.item() / logging_step:.3f}, acc = {train_acc / logging_step:.3f}")
                train_loss = train_acc = 0

    if validation:
        print("Evaluating Dev Set ...")
        model.eval()
        with torch.no_grad():
            dev_acc = 0
            for i, data in enumerate(tqdm(dev_loader)):
                output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
                # prediction is correct only if answer text exactly matches
                dev_acc += evaluate(data, output, i, 'dev') == dev_questions[i]["answer_text"]
            print(f"Validation | Epoch {epoch + 1} | acc = {dev_acc / len(dev_loader):.3f}")
        model.train()

# Save a model and its configuration file to the directory 「saved_model」 
# i.e. there are two files under the direcory 「saved_model」: 「pytorch_model.bin」 and 「config.json」
# Saved model can be re-loaded using 「model = BertForQuestionAnswering.from_pretrained("saved_model")」
print("Saving Model ...")
model_save_dir = "saved_model" 
model.save_pretrained(model_save_dir)

Start Training ...


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Epoch 1 | Step 100 | loss = 2.464, acc = 0.186
Epoch 1 | Step 200 | loss = 2.194, acc = 0.265
Epoch 1 | Step 300 | loss = 1.774, acc = 0.377
Epoch 1 | Step 400 | loss = 1.462, acc = 0.468
Epoch 1 | Step 500 | loss = 1.201, acc = 0.531
Epoch 1 | Step 600 | loss = 1.123, acc = 0.600
Epoch 1 | Step 700 | loss = 0.970, acc = 0.642
Epoch 1 | Step 800 | loss = 0.898, acc = 0.647
Epoch 1 | Step 900 | loss = 0.814, acc = 0.645
Epoch 1 | Step 1000 | loss = 0.819, acc = 0.654
Epoch 1 | Step 1100 | loss = 0.871, acc = 0.636
Epoch 1 | Step 1200 | loss = 0.780, acc = 0.676
Epoch 1 | Step 1300 | loss = 0.733, acc = 0.697
Epoch 1 | Step 1400 | loss = 0.766, acc = 0.671
Epoch 1 | Step 1500 | loss = 0.722, acc = 0.696
Epoch 1 | Step 1600 | loss = 0.677, acc = 0.735
Epoch 1 | Step 1700 | loss = 0.643, acc = 0.714
Epoch 1 | Step 1800 | loss = 0.644, acc = 0.734
Epoch 1 | Step 1900 | loss = 0.662, acc = 0.721
Epoch 1 | Step 2000 | loss = 0.689, acc = 0.719
Epoch 1 | Step 2100 | loss = 0.657, acc = 0.738
E

ID 32 Before : 周 王 是 燕 王 同 母 兄 弟 ， 而 朱 允 [UNK] 怕 他 與 燕 王 呵 成 一 氣
ID 32 After : 周王是燕王同母兄弟，而朱允炆怕他與燕王呵成一氣
ID 214 Before : 朝 鮮 王 朝 時 期 被 其 他 宗 教 取 代 。 韓 國 目 前 有 1100 多 萬 的 佛 教 信 徒 ， 其 中 90 % 的 信 徒 是 曹 溪 宗 。 儒 教 在 朝 鮮 三 國 時 期 從 中 傳 入 朝 鮮 半 島 。 百 濟 和 高 句 麗 的 教 育 制 度 都 是 以 儒 教 為 基 礎 。 高 句 麗 設 有 太 學 和 地 方 儒 學 私 立 學 院 [UNK] 堂 。 新 羅 統 一 三 國 後 ， 於 682 年 又 設 立 了 國 學 。 992 年 ， 高 麗 設 立 了 國 子 監 。 朝 鮮 王 朝 時 期
ID 214 After : 朝鮮王朝時期被其他宗教取代。韓國目前有1100多萬的佛教信徒，其中90%的信徒是曹溪宗。儒教在朝鮮三國時期從中傳入朝鮮半島。百濟和高句麗的教育制度都是以儒教為基礎。高句麗設有太學和地方儒學私立學院扃堂。新羅統一三國後，於682年又設立了國學。992年，高麗設立了國子監。朝鮮王朝時期
ID 305 Before : 大 肚 平 埔 族 拍 布 拉 族 大 肚 王 與 瑯 [UNK] 番 人 的 反 抗
ID 305 After : 大肚平埔族拍布拉族大肚王與瑯嶠番人的反抗
ID 305 Before : 大 肚 平 埔 族 拍 布 拉 族 大 肚 王 與 瑯 [UNK] 番 人 的 反 抗
ID 305 After : 大肚平埔族拍布拉族大肚王與瑯嶠番人的反抗
ID 529 Before : 《 阿 [UNK] 婆 吠 陀 》
ID 529 After : 《阿闥婆吠陀》
ID 716 Before : 姚 [UNK]
ID 716 After : 姚萇
ID 716 Before : 姚 [UNK]
ID 716 After : 姚萇
ID 869 Before : 光 緒 帝 在 康 有 為 、 梁 啓 超 等 人 的 鼓 動 下 ， 決 定 發 起 改 革 。 6 月 10 日 ， 光 緒 帝 令 翁 同 [UNK]
ID 869 After : 光緒帝

Epoch 2 | Step 100 | loss = 0.331, acc = 0.821
Epoch 2 | Step 200 | loss = 0.425, acc = 0.792
Epoch 2 | Step 300 | loss = 0.407, acc = 0.820
Epoch 2 | Step 400 | loss = 0.389, acc = 0.837
Epoch 2 | Step 500 | loss = 0.321, acc = 0.836
Epoch 2 | Step 600 | loss = 0.364, acc = 0.821
Epoch 2 | Step 700 | loss = 0.360, acc = 0.847
Epoch 2 | Step 800 | loss = 0.393, acc = 0.816
Epoch 2 | Step 900 | loss = 0.328, acc = 0.824
Epoch 2 | Step 1000 | loss = 0.395, acc = 0.806
Epoch 2 | Step 1100 | loss = 0.369, acc = 0.840
Epoch 2 | Step 1200 | loss = 0.332, acc = 0.827
Epoch 2 | Step 1300 | loss = 0.394, acc = 0.821
Epoch 2 | Step 1400 | loss = 0.344, acc = 0.812
Epoch 2 | Step 1500 | loss = 0.362, acc = 0.822
Epoch 2 | Step 1600 | loss = 0.339, acc = 0.822
Epoch 2 | Step 1700 | loss = 0.368, acc = 0.832
Epoch 2 | Step 1800 | loss = 0.372, acc = 0.842
Epoch 2 | Step 1900 | loss = 0.357, acc = 0.839
Epoch 2 | Step 2000 | loss = 0.371, acc = 0.830
Epoch 2 | Step 2100 | loss = 0.411, acc = 0.820
E

ID 84 Before : 《 進 步 日 報 》 。 4 月 14 日 ， 總 經 理 胡 政 之 病 逝 於 上 海 。 5 月 25 日 解 放 軍 占 領 上 海 ， 王 芸 生 等 從 解 放 區 趕 赴 上 海 。 5 月 29 日 ， 在 國 統 區 進 步 美 術 運 動 的 中 心 與 最 前 沿 的 上 海 ， 劉 開 渠 、 楊 可 揚 、 野 夫 、 朱 宣 咸 、 張 樂 平 、 龐 薰 [UNK] 、 趙 延 年 等 國 統 區 進 步 美 術 的 先 驅 們 代 表 以 上 海 為 中 心 的 國 統 區 進 步 美 術 力 量 在 《 大 公 報 》 發 表 迎 接 解 放 的 美 術 工 作 者 宣 言 ， 提 出 國 統 區 美 術 工 作 者 決 心 為 人 民 服 務 ， 依 照 新 民 主 主 義 所 指 示 的 目 標 ， 創 造 人 民 的 新 美 術 ， 該 宣 言 的 發 表 也 標 誌 著 國 統 區 美 術 史 和 上 海 近 代 美 術 史 從 此 翻 開 嶄 新 一 頁 。 6 月 17 日 ， 滬 版 發 表 《 大 公 報 新 生 宣 言 》 ， 宣 布 報 刊 歸 人 民 所 有 。 11 月 30 日 解 放 軍 占 領 重 慶 ， 渝 版 繼 續 出 版 ， 1952 年 8 月 4 日 終 刊 。 後 中 共 重 慶 市 委 在 其 基 礎 上 創 刊 市 委 機 關 報 《 重 慶 日 報 》 。 1953 年 1 月 1 日 ， 滬 版
ID 84 After : 改組為《進步日報》。4月14日，總經理胡政之病逝於上海。5月25日解放軍占領上海，王芸生等從解放區趕赴上海。5月29日，在國統區進步美術運動的中心與最前沿的上海，劉開渠、楊可揚、野夫、朱宣咸、張樂平、龐薰琹、趙延年等國統區進步美術的先驅們代表以上海為中心的國統區進步美術力量在《大公報》發表迎接解放的美術工作者宣言，提出國統區美術工作者決心為人民服務，依照新民主主義所指示的目標，創造人民的新美術，該宣言的發表也標誌著國統區美術史和上海近代美術史從此翻開嶄新一頁。6月17日，滬版發表《大公報新生宣言》，宣布報刊歸人民所有。11月30日解放軍占領重慶，渝版繼續出版，1952年8月4日終刊。後中共重慶市委在其基礎上創刊市委機關報《重慶日報

Epoch 3 | Step 100 | loss = 0.216, acc = 0.886
Epoch 3 | Step 200 | loss = 0.235, acc = 0.877
Epoch 3 | Step 300 | loss = 0.185, acc = 0.899
Epoch 3 | Step 400 | loss = 0.207, acc = 0.904
Epoch 3 | Step 500 | loss = 0.187, acc = 0.900
Epoch 3 | Step 600 | loss = 0.177, acc = 0.901
Epoch 3 | Step 700 | loss = 0.205, acc = 0.882
Epoch 3 | Step 800 | loss = 0.272, acc = 0.856
Epoch 3 | Step 900 | loss = 0.207, acc = 0.902
Epoch 3 | Step 1000 | loss = 0.198, acc = 0.891
Epoch 3 | Step 1100 | loss = 0.265, acc = 0.879
Epoch 3 | Step 1200 | loss = 0.219, acc = 0.890
Epoch 3 | Step 1300 | loss = 0.202, acc = 0.901
Epoch 3 | Step 1400 | loss = 0.205, acc = 0.886
Epoch 3 | Step 1500 | loss = 0.193, acc = 0.891
Epoch 3 | Step 1600 | loss = 0.199, acc = 0.886
Epoch 3 | Step 1700 | loss = 0.186, acc = 0.889
Epoch 3 | Step 1800 | loss = 0.222, acc = 0.891
Epoch 3 | Step 1900 | loss = 0.205, acc = 0.900
Epoch 3 | Step 2000 | loss = 0.198, acc = 0.890
Epoch 3 | Step 2100 | loss = 0.212, acc = 0.880
E

ID 32 Before : 周 王 是 燕 王 同 母 兄 弟 ， 而 朱 允 [UNK] 怕 他 與 燕 王 呵 成 一 氣
ID 32 After : 周王是燕王同母兄弟，而朱允炆怕他與燕王呵成一氣
ID 226 Before : 攻 擊 核 心 。 但 在 對 哥 倫 比 亞 之 戰 ， 內 馬 爾 被 對 手 哥 倫 比 亞 後 衞 朱 尼 嘉 膝 蓋 頂 到 背 部 ， 造 成 脊 椎 骨 骨 折 ， 隊 醫 隨 後 表 示 內 馬 爾 將 無 法 在 繼 續 接 下 來 的 比 賽 ， 以 及 隊 長 蒂 亞 戈 • 席 爾 瓦 被 [UNK] 停 賽 。 稍 後 的 準 決 賽 中 被 德 國 攻 進 7 球 ， 刷 新 了 巴 西 國 家 足 球 隊 的 最 大 比 數 失 利 記 錄 ， 這 場 賽 又 稱 為 米 內 羅 之 痛 ， 幾 日 後 的 季 軍 戰 中 又 以 0 - 3 輸 給 荷 蘭 隊 ， 再 次 刷 新 1950 年 主 辦 以 來 的 多 項 遠 古 紀 錄 ， 包 含 14 個 失 球 。 賽 後 大 部 人 批 評 巴 西 踢 法 太 英 雄 主 義 及 個 人 主 義 ， 再 加 上 缺 少 內 馬 爾 般 的 世 界 級 球 員
ID 226 After : 攻擊核心。但在對哥倫比亞之戰，內馬爾被對手哥倫比亞後衞朱尼嘉膝蓋頂到背部，造成脊椎骨骨折，隊醫隨後表示內馬爾將無法在繼續接下來的比賽，以及隊長蒂亞戈•席爾瓦被罸停賽。稍後的準決賽中被德國攻進7球，刷新了巴西國家足球隊的最大比數失利記錄，這場賽又稱為米內羅之痛，幾日後的季軍戰中又以0-3輸給荷蘭隊，再次刷新1950年主辦以來的多項遠古紀錄，包含14個失球。賽後大部人批評巴西踢法太英雄主義及個人主義，再加上缺少內馬爾般的世界級球員
ID 305 Before : 大 肚 平 埔 族 拍 布 拉 族 大 肚 王 與 瑯 [UNK] 番 人 的 反 抗
ID 305 After : 大肚平埔族拍布拉族大肚王與瑯嶠番人的反抗
ID 305 Before : 大 肚 平 埔 族 拍 布 拉 族 大 肚 王 與 瑯 [UNK] 番 人 的 反 抗
ID 305 After : 大肚平埔族拍布拉族大肚王與瑯嶠番人的反抗
ID 529 Before : 《 阿 [UNK] 婆 

In [15]:
# Postprocessing

def postprocessing(result, index):
    if result:
        if result[0] == '，' or result[0] == '。':
            print(f'ID {index} Before : {result}')
            result = result[1:]
            print(f'ID {index} After  : {result}')

        if  result.find('《') != -1 and result.find('》') == -1:
            print(f'ID {index} Before : {result}')
            result = result + '》'
            print(f'ID {index} After  : {result}')
        
        if result.find('《') == -1 and result.find('》') != -1:
            print(f'ID {index} Before : {result}')
            result = '《' + result  
            print(f'ID {index} After  : {result}')

        if  result.find('「') != -1 and result.find('」') == -1:
            print(f'ID {index} Before : {result}')
            result = result + '」'
            print(f'ID {index} After  : {result}')
        
        if result.find('「') == -1 and result.find('」') != -1:
            print(f'ID {index} Before : {result}')
            result = '「' + result  
            print(f'ID {index} After  : {result}')
        
        
    return result

## Testing

In [16]:
print("Evaluating Test Set ...")

result = []

model.eval()
with torch.no_grad():
    for i, data in enumerate(tqdm(test_loader)):
        output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        result.append(evaluate(data, output, i, 'test'))

result_file = "result.csv"
with open(result_file, 'w') as f:	
	  f.write("ID,Answer\n")
	  for i, test_question in enumerate(test_questions):
        # Replace commas in answers with empty strings (since csv is separated by comma)
        # Answers in kaggle are processed in the same way
                result[i] = postprocessing(result[i], i)
                f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

print(f"Completed! Result is in {result_file}")

Evaluating Test Set ...


ID 159 Before : 哪 一 位? [SEP] 可 汗 或 大 汗 ， 又 譯 為 可 寒 、 合 罕 ， 通 黑 汗 ， 統 治 者 頭 銜 ， 為 部 落 最 高 領 袖 或 皇 帝 之 意 ， 廣 泛 使 用 於 蒙 古 與 中 亞 地 區 。 古 代 北 亞 遊 牧 民 族 鮮 卑 、 回 [UNK] 、 柔 然 、 高 車 、 突 厥 、 吐 谷 渾 、 鐵 勒 、 女 真 等 建 立 的 汗 國 ， 其 君 主 或 最 高 政 治 首 領 皆 稱 可 汗 或 大 汗 。 可 汗 為 汗 國 的 統 治 者 ， 這 個 稱 呼 最 早 是 蒙 古 與 突 厥 等 遊 牧 部 落 中 ， 對 首 領 的 尊 稱 ， 原 意 王 朝 、 神 靈 和 上 天 ， 類 似 漢 語 所 說 的 天 子 ， 或 是 皇 帝 。 可 汗 最 早 的 記 錄 出 現 於 3 世 紀 ， 為 鮮 卑 部 落 官 稱 ， 據 《 宋 書 》 記 載 乙 那 婁 與 慕 容 吐 谷 渾 對 話 ， 稱 吐 谷 渾 為 可 寒 此 外 著 名 的 北 魏 太 武 帝
ID 159 After : 
ID 250 Before : 溥 [UNK]
ID 250 After : 溥儁
ID 250 Before : 溥 [UNK]
ID 250 After : 溥儁
ID 332 Before : 目 前 沒 有 觀 察 到 任 何 語 言 純 [UNK] 以 力 道 來 區 分 不 同 輔 音
ID 332 After : 目前沒有觀察到任何語言純綷以力道來區分不同輔音
ID 332 Before : 目 前 沒 有 觀 察 到 任 何 語 言 純 [UNK] 以 力 道 來 區 分 不 同 輔 音
ID 332 After : 目前沒有觀察到任何語言純綷以力道來區分不同輔音
ID 340 Before : [UNK] 人 國
ID 340 After : 荇人國
ID 563 Before : 馬 [UNK]
ID 563 After : 馬馼
ID 563 Before : 馬 [UNK]
ID 563 After : 馬馼
ID 635 Before : 東 晉 常 [UNK]
ID 635 After : 東晉常璩
ID 892 Before : 周 恩 來 、 

In [17]:
from google.colab import files
files.download("result.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>